In [45]:
!pip install numpy
!pip install matplotlib
!pip install pandas

import os

SIRPath = "/home/het/Documents/github-projects/SIR-MODEL"
print(os.getcwd())
os.chdir(SIRPath)
print(os.getcwd())

import requests 
from sys import platform
import pandas as pd 
import numpy as np 
from numpy.polynomial import Chebyshev
from matplotlib import pyplot as plt 
import json
from sys import platform

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
/home/het/Documents/github-projects/SIR-MODEL
/home/het/Documents/github-projects/SIR-MODEL


In [46]:
## IsWindows defintion

def isWindows():
    return platform.lower().startswith("win")

In [47]:
## urlwrite function.
print(os.getcwd())
def urlwrite(url, fileName):
    response = requests.get(url)

    for i in range(5):
        if response.status_code == 200:
            break
        response = requests.get(url)
    else:
        raise Exception("Internet not working")
    
    with open(fileName, 'w') as fptr:
        fptr.write(response.text)


/home/het/Documents/github-projects/SIR-MODEL


In [48]:
## Required constants

dummyFileName = 'data\dummy.csv' if isWindows() else 'data/dummy.csv'
DateFormat = '%Y-%m-%d'
fileName = 'data\cityAndStateNames.json' if isWindows() else 'data/cityAndStateNames.json'

In [49]:
## Get district names and get state names
def getDistrictNames():
    with open(fileName) as fptr:
        return json.loads(fptr.read())['cityNames']

def getStateNames():
    with open(fileName) as fptr:
        return json.loads(fptr.read())['stateNames']

In [50]:
## Movmean definition 

def movmean(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [51]:
## Init class for init element

class Init:
    def __init__(self, country, date, C, D, R, Npop):
        self.country = country
        self.date = date
        self.C = C
        self.D = D
        self.R = R
        self.Npop = Npop

In [52]:
ifplotPR = True
dists = True
debug = False
ifprint = True
span = 7

In [53]:
fullName, sb_number,  name = None, None, None 

In [54]:
if dists:
    fullName = 'https://api.covid19india.org/csv/latest/districts.csv'
    name = getDistrictNames()
    tt1 = pd.to_datetime('2020-4-26')
    sb_number = 4
else:
    fullName = 'https://api.covid19india.org/csv/latest/states.csv'
    name = getStateNames()
    tt1 = pd.to_datetime('2020-3-14')
    sb_number = 5

In [55]:
urlwrite(fullName, dummyFileName)

tableAll = pd.read_csv(dummyFileName, na_values={'Tested':['']})
tableAll.Date = pd.to_datetime(tableAll.Date, format="%Y-%m-%d")

span1 = 14

mean_si, min_mean_si, max_mean_si = 4.7, 3.7, 6.0
std_si, min_std_si, max_std_si = 2.9, 1.9, 4.9 

In [56]:
for n in range(len(name)):

    Location = name[n]

    # To avoid scope related problems.
    tableLocation = None
    R, D, C, O, T = None, None, None, None, None

    dateInit = pd.to_datetime('2020-03-01', format = DateFormat)

    if dists:
        tableLocation = tableAll[(tableAll.District == Location) & (tableAll.Date >= pd.to_datetime('1-3-2020', format='%d-%m-%Y'))]

        R = tableLocation['Recovered'].to_numpy()
        D = tableLocation['Deceased'].to_numpy()
        C = tableLocation['Confirmed'].to_numpy()
        O = tableLocation['Other'].to_numpy()
        T = tableLocation['Tested'].to_numpy()
    else:
        tableLocation = tableAll[(tableAll.State == Location) & (tableAll.Date >= pd.to_datetime('1-3-2020', format='%d-%m-%Y'))]

        R = tableLocation['Recovered'].to_numpy()
        D = tableLocation['Deceased'].to_numpy()
        C = tableLocation['Confirmed'].to_numpy()
        O = tableLocation['Other'].to_numpy()
        T = tableLocation['Tested'].to_numpy()
 
    date = tableLocation['Date'].to_numpy()
    A = C - D - O - R # Active cases

    dailyC = np.diff(C)

    if debug:
        fig = plt.figure()
        fig.add_subplot(2, 1, (1))

        plt.stem(date, C)
        plt.xlabel('Date')
        plt.ylabel('Confirmed Cases')
            
        fig.add_subplot(2, 1, (2))

        plt.stem(date[1:], dailyC)
        plt.xlabel('Date')
        plt.ylabel('Infection rate')

        plt.show()

    PR = (np.diff(movmean(C,span)) / np.diff(movmean(T,span))) *100
    CFR = (np.diff(movmean(D,span)) / np.diff(movmean(C,span))) *100
    Testing = np.diff(T)

    date0 = np.copy(date)
    tt1 = None

    if dists:
        tt1 = str(date0[0])
        sb_number = 4
    else:
        tt1 = str(date0[0])
        sb_number = 5
        
    date1 = pd.to_datetime('2021-4-1', format = DateFormat)

    dateEnd = date[-1]

    tableLocation1 = tableLocation[(tableLocation.Date > date1) & (tableLocation.Date < dateEnd)]

    date = tableLocation1['Date'].to_numpy()
    C = tableLocation1['Confirmed'].to_numpy()
    D = tableLocation1['Deceased'].to_numpy()
    R = tableLocation1['Recovered'].to_numpy()
    Npop = int('80e6', 16)

    init = Init(Location, date, C, D, R, Npop)
    # Init is a simple class to save data like a struct object.

    diffC = np.diff(init.C)

    q1 = diffC[0]
    q2 = diffC[-1]
    t = diffC.shape[0]

    Cexp = np.copy(init.C)
    texp = list(range(1, Cexp.shape[0] + 1))

    returnVal = Chebyshev.fit(texp, Cexp, deg = 2)

    print(returnVal)            

860622.7886016696 + 467038.849074976·T₁(x) - 77057.46557343165·T₂(x)
856789.1059746227 + 249947.45034079862·T₁(x) - 71184.21734268502·T₂(x)
626388.2627439075 + 124402.92015579376·T₁(x) - 52744.898819857255·T₂(x)
496919.95005933556 + 103145.89678675757·T₁(x) - 41847.507873356786·T₂(x)
403182.2291401085 + 132185.55501460575·T₁(x) - 55013.70589278623·T₂(x)
234872.40386336998 + 126568.52969815001·T₁(x) - 5461.989671549053·T₂(x)
170284.91540255214 + 99064.982473223·T₁(x) - 29499.706054937706·T₂(x)
185755.7135143077 + 79105.57838364174·T₁(x) - 37780.481893080876·T₂(x)
314729.1041625295 + 111832.77653359297·T₁(x) - 32518.376922996766·T₂(x)
216566.61395247717 + 102708.26144109062·T₁(x) - 10836.27376260375·T₂(x)
131237.98561055452 + 79717.55988315487·T₁(x) - 13463.929622238991·T₂(x)
394956.14170160575 + 169897.9834469329·T₁(x) - 6680.9446470782395·T₂(x)
131945.12604817943 + 72691.29016553066·T₁(x) - 19945.522104654552·T₂(x)
107464.97032747399 + 56221.08617332037·T₁(x) - 3353.8130733357057·T₂(x)

IndexError: index 0 is out of bounds for axis 0 with size 0